In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Configuration
test_config = {
    'tickers': ['NVDA', 'AMD', 'INTC'],
    'weights': [0.4, 0.4, 0.2],
    'start_date': '2023-01-01',
    'end_date': '2024-01-01',
    'alpaca_key_id': os.getenv('ALPACA_KEY_ID'),
    'alpaca_secret_key': os.getenv('ALPACA_SECRET_KEY'),
    'paper_trading': True
}

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = [15, 10]

try:
    # 1. Initialize Configuration
    print("Configuration:")
    print("-" * 50)
    for key, value in test_config.items():
        if 'secret' not in key.lower():  # Don't print sensitive info
            print(f"{key}: {value}")
    
    # 2. Load Data
    portfolio_config = PortfolioConfig(**test_config)
    loader = DataLoader(portfolio_config)
    market_data = loader.load_stocks()
    
    # 3. Print Data Summary
    print("\nData Summary:")
    print("=" * 50)
    for ticker in test_config['tickers']:
        returns = market_data['returns'][ticker]
        print(f"\n{ticker}:")
        print(f"Annual Return: {returns.mean() * 252:.1%}")
        print(f"Annual Volatility: {returns.std() * np.sqrt(252):.1%}")
    
    print("\nData Loading Successful!")
    print("-" * 50)
    print("\nShape of data:")
    print(f"Close prices: {market_data['close'].shape}")
    print(f"Returns: {market_data['returns'].shape}")
    
    # 4. Visualizations
    # Plot 1: Normalized Prices
    plt.figure(figsize=(15, 8))
    for ticker in test_config['tickers']:
        plt.plot(
            market_data['close'].index,
            market_data['close'][ticker] / market_data['close'][ticker].iloc[0],
            label=ticker,
            linewidth=2
        )
    plt.title('Normalized Stock Prices', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Price (Normalized)', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Plot 2: Rolling Volatility
    plt.figure(figsize=(15, 8))
    window = 21  # 21-day rolling window
    for ticker in test_config['tickers']:
        rolling_vol = market_data['returns'][ticker].rolling(window).std() * np.sqrt(252)
        plt.plot(rolling_vol.index, rolling_vol, label=f'{ticker} ({window}-day)', linewidth=2)
    plt.title(f'{window}-Day Rolling Volatility', fontsize=14)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Annualized Volatility', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Plot 3: Correlation Matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        market_data['returns'].corr(),
        annot=True,
        cmap='RdYlBu',
        center=0,
        vmin=-1,
        vmax=1,
        square=True
    )
    plt.title('Correlation Matrix', fontsize=14)
    plt.show()

    # 5. Summary Statistics
    stats = loader.get_summary_statistics(market_data)
    print("\nPortfolio Statistics:")
    print("=" * 50)
    print(f"Annual Return: {stats['returns']['mean']:.2%}")
    print(f"Annual Volatility: {stats['returns']['std']:.2%}")
    print(f"Skewness: {stats['returns']['skew']:.2f}")
    print(f"Kurtosis: {stats['returns']['kurt']:.2f}")
    print("\nAverage Daily Volume:")
    for ticker in test_config['tickers']:
        print(f"{ticker}: {stats['daily_volume_avg'][ticker]:,.0f}")

except Exception as e:
    print(f"\nError: {str(e)}")
    raise
